In [66]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [67]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [68]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-CxjuAQUwdNIIvYqRSWhvpXojy7wAU',
 'object': 'chat.completion',
 'created': 1768353846,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: Honey never spoils—archaeologists have found edible honey in ancient Egyptian tombs over 3,000 years old. Want another?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 872,
  'total_tokens': 883,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 832,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [69]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: Honey never spoils—archaeologists have found edible honey in ancient Egyptian tombs over 3,000 years old. Want another?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [70]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Fun fact: On Venus, a day is longer than a year — it takes about 243 Earth days to rotate once, but only about 225 Earth days to orbit the Sun. And Venus spins the opposite way, so the Sun rises in the west.'

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [71]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [72]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [73]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [74]:
response = ollama.chat.completions.create(model="llama3.2", messages= [{ "role":"user", "content":"Tell me a fun fact about Iran" }])
response.choices[0].message.content

"Here's one:\n\nIran is home to the world's oldest known bridge, the Khujut-Reis Bridge, which is over 2,500 years old! Built during the Sassanian Empire (224-651 CE), this ancient bridge in Yazd, Iran, spans a dry river bed and is an extraordinary example of ancient engineering.\n\nIsn't that cool?"

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [90]:
import os
import requests
from openai import OpenAI
from web_scraper import fetch_website_contents
from IPython.display import Markdown, display

In [91]:
openai = OpenAI()

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [92]:
system_prompt = """
You are a helpful assistant that analyzes the contents of a website,
and provides a short, helpful, comprehensive summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.
"""

In [93]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website} ]

In [94]:
def summarize(url):
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model="llama3.2", messages=messages_for(website)
        )
    return response.choices[0].message.content

In [95]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [96]:
display_summary("https://cnn.com")

**Summary of the Website**

The website is a news and information portal from CNN (Cable News Network), providing users with up-to-date breaking news, commentary, analysis, photos, videos, and entertainment content. The site covers various topics such as politics, business, entertainment, sports, science, health, and climate. The website also includes features like ads for relevant products, quizzes, puzzles, games, and subscriptions to newsletters.

**News and Announcements**

Recent news articles and announcements cover the following stories:

* Breaking News: Latest on Iran and Venezuela protests
	+ Trump's response to Iran's civil unrest, with calls from the US to evacuate citizens
	+ Venezuelan opposition candidate Juan Guaido claims Maduro's "dictatorial regime" is collapsing
	+ Protesters in Hong Kong display their determination to fight for democracy against China
* Health and Wellness
	+ A COVID-19 vaccine may not offer significant protection in the elderly population: study suggests limited age-restriction impact on the effectiveness level of current mRNA/Chad Johnson vaccination regimens
	+ Global obesity rates declining according to a new World Health Organization (WHO) report
	+ New evidence raises hope for potentially more effective HIV treatment strategies

Note that this summary mainly focuses on news and announcements from the website. The full site contains additional information, including articles from various authors, reviews of products and services, personal profiles, podcasts, and videos.

Please note the limitations of summarizing a dynamic content like websites - it makes sense to consider only key headlines and relevant subtopics.



Please let me know if you would like any further assistance with summarizing your website.